In [ ]:
import os
import glob

import contextily as ctx
import geopandas as gpd
import imageio
import matplotlib.pyplot as plt
import matplotlib.colors
import pandas as pd
import pygifsicle

In [ ]:
flows = pd.read_csv(os.path.join('data','link_use_central_oxford.csv')) \
    .rename(columns={'edgeID': 'link', 'vehicles': 'flow'})
flows.head()

In [ ]:
flow_max = flows.flow.max()
flow_max

In [ ]:
unique_link_ids = flows.link.unique()
unique_link_ids

In [ ]:
len(unique_link_ids)

In [ ]:
links = gpd.read_file(os.path.join('data','fullNetworkWithEdgeIDs.shp')) \
    .drop(columns=['Anode', 'Bnode', 'CP', 'iDir', 'SRefE', 'SRefN', 'Distance', 'FFspeed', 'FFtime', 'IsFerry']) \
    .rename(columns={'EdgeID': 'link'})
links.head()

In [ ]:
hourly_maxes = flows.drop(columns=['link']).groupby('hour').max()
hourly_maxes.head()

In [ ]:
def plot_map(hour, links, flow_max, output_filename):
    ax = links.plot(
        figsize=(10, 12),
        column='flow',
        cmap='YlOrRd',
        norm=matplotlib.colors.Normalize(vmin=0.0001, vmax=flow_max),
        legend=True,
        legend_kwds={'label': 'Flow volume', 'orientation': 'horizontal'}
    )
    plt.title('{:02d}:00'.format(hour), fontsize=16)
    ctx.add_basemap(ax, crs=links.crs)
    plt.savefig(output_filename)
    plt.close()

In [ ]:
hours = [
    'MIDNIGHT',
    'ONEAM',
    'TWOAM',
    'THREEAM',
    'FOURAM',
    'FIVEAM',
    'SIXAM',
    'SEVENAM',
    'EIGHTAM',
    'NINEAM',
    'TENAM',
    'ELEVENAM',
    'NOON',
    'ONEPM',
    'TWOPM',
    'THREEPM',
    'FOURPM',
    'FIVEPM',
    'SIXPM',
    'SEVENPM',
    'EIGHTPM',
    'NINEPM',
    'TENPM',
    'ELEVENPM',
]
for hour, hour_key in enumerate(hours):
    hour_flows = flows[flows.hour == hour_key].copy()
    hour_flows = links.merge(hour_flows, on='link')
    plot_map(hour, hour_flows, flow_max, os.path.join('vis', '{:02d}.png'.format(hour)))

In [ ]:
gif_path = os.path.join('vis', 'movie.gif')
filenames = glob.glob(os.path.join('vis','*.png'))

images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave(os.path.join(gif_path), images)

In [ ]:
pygifsicle.optimize(gif_path)